# **Movers Dashboard: [Streamlit](https://streamlit.io/)**
### *Kathleen created 5/30/25; last edited 6/13/25*

## **Step 1: Structure the Data for Visualizing**

In [110]:
# Import packages
import pandas as pd # for data manipulation
import numpy as np # for recoding variables
import streamlit as st # provides the web app framework
import plotly.express as px # for choropleth maps

In [111]:
# Load data prepared by Ahmed
df_inter_state = pd.read_csv('df_inter_state.csv')

df_inter_state["current_state_code"] = df_inter_state["state_code"] # Rename column for clarity

In [112]:
# Keep just the columns needed for visualization
df_inter_state = df_inter_state[["current_state", "previous_state", "sex", "age_group", "education", "marital_status", "person_weight"]]

df_inter_state = df_inter_state.dropna() # Drop rows with NaN values (there shouldn't be any, but just in case...)

In [113]:
# Recode education with fewer categories
df_inter_state["education"] = np.where((df_inter_state["education"] == "No schooling") | (df_inter_state["education"] == "Elementary/Middle school") | (df_inter_state["education"] == "Some high school"), "Less than High School",
                                    np.where(df_inter_state["education"] == "High school graduate", "High School Grad",
                                    np.where((df_inter_state["education"] == "Some college") | (df_inter_state["education"] == "Associate's degree"), "Some College",
                                    np.where(df_inter_state["education"] == "Bachelor's degree", "Bachelor's Degree", "Graduate Degree"))))

# Recode marital_status with fewer categories
df_inter_state["marital_status"] = np.where((df_inter_state["marital_status"] == "Divorced") | (df_inter_state["marital_status"] == "Separated"), "Divorced/Separated", df_inter_state["marital_status"])

# Create ordered categories
age_order = ["0-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"]
df_inter_state["age_group"] = pd.Categorical(df_inter_state["age_group"], categories=age_order, ordered=True)

education_order = ["Less than High School", "High School Grad", "Some College", "Bachelor's Degree", "Graduate Degree"]
df_inter_state["education"] = pd.Categorical(df_inter_state["education"], categories=education_order, ordered=True)

mar_order = ["Never Married", "Married", "Divorced/Separated", "Widowed"]
df_inter_state["marital_status"] = pd.Categorical(df_inter_state["marital_status"], categories=mar_order, ordered=True)

## **Step 1A: Create an INBOUND movers dataset**

In [114]:
# Get counts grouped by relevant columns for INBOUND migration
inbound = df_inter_state.groupby(["current_state", "sex", "age_group", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound

,current_state,sex,age_group,education,marital_status,count
0,Alabama,Female,0-17,Less than High School,Never Married,9920
1,Alabama,Female,0-17,Less than High School,Married,0
2,Alabama,Female,0-17,Less than High School,Divorced/Separated,0
3,Alabama,Female,0-17,Less than High School,Widowed,0
4,Alabama,Female,0-17,High School Grad,Never Married,0
...,...,...,...,...,...,...
14275,Wyoming,Male,65+,Bachelor's Degree,Widowed,0
14276,Wyoming,Male,65+,Graduate Degree,Never Married,0
14277,Wyoming,Male,65+,Graduate Degree,Married,0
14278,Wyoming,Male,65+,Graduate Degree,Divorced/Separated,77


In [115]:
# Create "all" counts for each variable 
# Make sure to include all combinations of the variables! (there is probably a more efficient way to do this, haha)

inbound_sex_all = df_inter_state.groupby(["current_state","age_group", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound_sex_all["sex"] = "All"

inbound_age_all = df_inter_state.groupby(["current_state","sex", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound_age_all["age_group"] = "All"

inbound_ed_all = df_inter_state.groupby(["current_state","sex", "age_group", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound_ed_all["education"] = "All"

inbound_mar_all = df_inter_state.groupby(["current_state","sex", "age_group", "education"])["person_weight"].sum().reset_index(name="count")
inbound_mar_all["marital_status"] = "All"

inbound_sa_all = df_inter_state.groupby(["current_state", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound_sa_all["sex"] = "All"
inbound_sa_all["age_group"] = "All"

inbound_se_all = df_inter_state.groupby(["current_state", "age_group", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound_se_all["sex"] = "All"
inbound_se_all["education"] = "All"

inbound_sm_all = df_inter_state.groupby(["current_state", "age_group", "education"])["person_weight"].sum().reset_index(name="count")
inbound_sm_all["sex"] = "All"
inbound_sm_all["marital_status"] = "All"

inbound_ae_all = df_inter_state.groupby(["current_state", "sex", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound_ae_all["age_group"] = "All"
inbound_ae_all["education"] = "All"

inbound_am_all = df_inter_state.groupby(["current_state", "sex", "education"])["person_weight"].sum().reset_index(name="count")
inbound_am_all["age_group"] = "All"
inbound_am_all["marital_status"] = "All"

inbound_em_all = df_inter_state.groupby(["current_state", "sex", "age_group"])["person_weight"].sum().reset_index(name="count")
inbound_em_all["education"] = "All"
inbound_em_all["marital_status"] = "All"

inbound_sae_all = df_inter_state.groupby(["current_state", "marital_status"])["person_weight"].sum().reset_index(name="count")
inbound_sae_all["sex"] = "All"
inbound_sae_all["age_group"] = "All"
inbound_sae_all["education"] = "All"

inbound_sam_all = df_inter_state.groupby(["current_state", "education"])["person_weight"].sum().reset_index(name="count")
inbound_sam_all["sex"] = "All"
inbound_sam_all["age_group"] = "All"
inbound_sam_all["marital_status"] = "All"

inbound_sem_all = df_inter_state.groupby(["current_state", "age_group"])["person_weight"].sum().reset_index(name="count")
inbound_sem_all["sex"] = "All"
inbound_sem_all["education"] = "All"
inbound_sem_all["marital_status"] = "All"

inbound_aem_all = df_inter_state.groupby(["current_state", "sex"])["person_weight"].sum().reset_index(name="count")
inbound_aem_all["age_group"] = "All"
inbound_aem_all["education"] = "All"
inbound_aem_all["marital_status"] = "All"

inbound_saem_all = df_inter_state.groupby(["current_state"])["person_weight"].sum().reset_index(name="count")
inbound_saem_all["sex"] = "All"
inbound_saem_all["age_group"] = "All"
inbound_saem_all["education"] = "All"
inbound_saem_all["marital_status"] = "All"

inbound = pd.concat([inbound_saem_all, inbound_aem_all, inbound_sem_all, inbound_sam_all, inbound_sae_all, inbound_em_all, inbound_ae_all, inbound_sm_all, inbound_se_all,
                      inbound_sa_all, inbound_mar_all, inbound_ed_all, inbound_age_all, inbound_sex_all, inbound])

In [116]:
# Merge in state abbreviations needed to create choropleth maps
url ="https://www2.census.gov/geo/docs/reference/state.txt"
states = pd.read_csv(url, sep='|')

states["current_state"] = states["STATE_NAME"] # Rename column to merge on
states["current_fips"] = states["STATE"] # Rename column for clarity
states["current_state_code"] = states["STUSAB"] # Rename column for clarity

inbound_states = states[["current_state", "current_state_code", "current_fips"]] # Keep only the columns needed

inbound = pd.merge(inbound,inbound_states,on="current_state",how="left") # execute the merge

inbound = inbound.dropna() # Drop rows with NaN values, just in case there are any

inbound


,current_state,count,sex,age_group,education,marital_status,current_state_code,current_fips
0,Alabama,120307,All,All,All,All,AL,1
1,Alaska,35633,All,All,All,All,AK,2
2,Arizona,250726,All,All,All,All,AZ,4
3,Arkansas,75745,All,All,All,All,AR,5
4,California,425194,All,All,All,All,CA,6
...,...,...,...,...,...,...,...,...
36205,Wyoming,0,Male,65+,Bachelor's Degree,Widowed,WY,56
36206,Wyoming,0,Male,65+,Graduate Degree,Never Married,WY,56
36207,Wyoming,0,Male,65+,Graduate Degree,Married,WY,56
36208,Wyoming,77,Male,65+,Graduate Degree,Divorced/Separated,WY,56


In [117]:
# Save to CSV
inbound.to_csv('inbound.csv', index=False)

In [118]:
# Test the choropleth map code with a specific set of filters
filtered_df = inbound[(inbound["sex"] == "Female") & (inbound["age_group"] == "0-17") & (inbound["education"] == "Less than High School") & (inbound["marital_status"] == "Never Married")]

choropleth = px.choropleth(filtered_df, locations="current_state_code", color="count", locationmode="USA-states",color_continuous_scale="reds",
                               scope="usa",
                              )

choropleth.show()

## **Step 1B: Create an OUTBOUND movers dataset**

In [119]:
# Get counts grouped by relevant columns for OUTBOUND migration
outbound = df_inter_state.groupby(["previous_state", "sex", "age_group", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")

In [120]:
# Create "all" counts for each variable 
# Make sure to include all combinations of the variables! (there is probably a more efficient way to do this, haha)

outbound_sex_all = df_inter_state.groupby(["previous_state","age_group", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")
outbound_sex_all["sex"] = "All"

outbound_age_all = df_inter_state.groupby(["previous_state","sex", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")
outbound_age_all["age_group"] = "All"

outbound_ed_all = df_inter_state.groupby(["previous_state","sex", "age_group", "marital_status"])["person_weight"].sum().reset_index(name="count")
outbound_ed_all["education"] = "All"

outbound_mar_all = df_inter_state.groupby(["previous_state","sex", "age_group", "education"])["person_weight"].sum().reset_index(name="count")
outbound_mar_all["marital_status"] = "All"

outbound_sa_all = df_inter_state.groupby(["previous_state", "education", "marital_status"])["person_weight"].sum().reset_index(name="count")
outbound_sa_all["sex"] = "All"
outbound_sa_all["age_group"] = "All"

outbound_se_all = df_inter_state.groupby(["previous_state", "age_group", "marital_status"])["person_weight"].sum().reset_index(name="count")
outbound_se_all["sex"] = "All"
outbound_se_all["education"] = "All"

outbound_sm_all = df_inter_state.groupby(["previous_state", "age_group", "education"])["person_weight"].sum().reset_index(name="count")
outbound_sm_all["sex"] = "All"
outbound_sm_all["marital_status"] = "All"

outbound_ae_all = df_inter_state.groupby(["previous_state", "sex", "marital_status"])["person_weight"].sum().reset_index(name="count")
outbound_ae_all["age_group"] = "All"
outbound_ae_all["education"] = "All"

outbound_am_all = df_inter_state.groupby(["previous_state", "sex", "education"])["person_weight"].sum().reset_index(name="count")
outbound_am_all["age_group"] = "All"
outbound_am_all["marital_status"] = "All"

outbound_em_all = df_inter_state.groupby(["previous_state", "sex", "age_group"])["person_weight"].sum().reset_index(name="count")
outbound_em_all["education"] = "All"
outbound_em_all["marital_status"] = "All"

outbound_sae_all = df_inter_state.groupby(["previous_state", "marital_status"])["person_weight"].sum().reset_index(name="count")
outbound_sae_all["sex"] = "All"
outbound_sae_all["age_group"] = "All"
outbound_sae_all["education"] = "All"

outbound_sam_all = df_inter_state.groupby(["previous_state", "education"])["person_weight"].sum().reset_index(name="count")
outbound_sam_all["sex"] = "All"
outbound_sam_all["age_group"] = "All"
outbound_sam_all["marital_status"] = "All"

outbound_sem_all = df_inter_state.groupby(["previous_state", "age_group"])["person_weight"].sum().reset_index(name="count")
outbound_sem_all["sex"] = "All"
outbound_sem_all["education"] = "All"
outbound_sem_all["marital_status"] = "All"

outbound_aem_all = df_inter_state.groupby(["previous_state", "sex"])["person_weight"].sum().reset_index(name="count")
outbound_aem_all["age_group"] = "All"
outbound_aem_all["education"] = "All"
outbound_aem_all["marital_status"] = "All"

outbound_saem_all = df_inter_state.groupby(["previous_state"])["person_weight"].sum().reset_index(name="count")
outbound_saem_all["sex"] = "All"
outbound_saem_all["age_group"] = "All"
outbound_saem_all["education"] = "All"
outbound_saem_all["marital_status"] = "All"

outbound = pd.concat([outbound_saem_all, outbound_aem_all, outbound_sem_all, outbound_sam_all, outbound_sae_all, outbound_em_all, outbound_ae_all, outbound_sm_all, outbound_se_all,
                      outbound_sa_all, outbound_mar_all, outbound_ed_all, outbound_age_all, outbound_sex_all, outbound])

In [121]:
# Merge in state abbreviations needed to create choropleth maps
url ="https://www2.census.gov/geo/docs/reference/state.txt"
states = pd.read_csv(url, sep='|')

states["previous_state"] = states["STATE_NAME"] # Rename column to merge on
states["previous_fips"] = states["STATE"] # Rename column for clarity
states["previous_state_code"] = states["STUSAB"] # Rename column for clarity

outbound_states = states[["previous_state", "previous_state_code", "previous_fips"]] # Keep only the columns needed

outbound = pd.merge(outbound,outbound_states,on="previous_state",how="left") # Execute the merge

outbound = outbound.dropna() # Drop rows with NaN values, just in case there are any

outbound

,previous_state,count,sex,age_group,education,marital_status,previous_state_code,previous_fips
0,Alabama,101870,All,All,All,All,AL,1
1,Alaska,36206,All,All,All,All,AK,2
2,Arizona,193757,All,All,All,All,AZ,4
3,Arkansas,65211,All,All,All,All,AR,5
4,California,682598,All,All,All,All,CA,6
...,...,...,...,...,...,...,...,...
36915,Wyoming,0,Male,65+,Bachelor's Degree,Widowed,WY,56
36916,Wyoming,0,Male,65+,Graduate Degree,Never Married,WY,56
36917,Wyoming,78,Male,65+,Graduate Degree,Married,WY,56
36918,Wyoming,0,Male,65+,Graduate Degree,Divorced/Separated,WY,56


In [122]:
# Save to CSV
outbound.to_csv('outbound.csv', index=False)

In [123]:
# Test the choropleth map code with a specific set of filters
filtered_df2 = outbound[(outbound["sex"] == "All") & (outbound["age_group"] == "All") & (outbound["education"] == "All") & (outbound["marital_status"] == "All")]

choropleth = px.choropleth(filtered_df2, locations="previous_state_code", color="count", locationmode="USA-states",color_continuous_scale="reds",
                               scope="usa",
                              )

choropleth.show()

## **Step 2: Create the Streamlit Page Configuration File**
### The code chunk below is commented out because it should be saved and run in a separate .py file. It is included below for convenience/informational purposes.
### After running the .py file, you will see some warning/thread messages. You can disregard them. There will also be a message like, *"Warning: to view this Streamlit app on a browser, run it with the following command: **streamlit run file_location/file_name.py [ARGUMENTS]**"* Copy and paste that command into your Terminal application to run the dashboard app locally.
### To deploy a dashboard app publicly, follow instructions [here](https://docs.streamlit.io/deploy/streamlit-community-cloud/deploy-your-app).
### The dashboard created with this code can be found publicly [here](https://a3moversdash.streamlit.app/). If you see a message saying the "app has gone to sleep," click "Yes, get this app back up!"

In [124]:
# #######################
# # DSTP Cohort 6 Capstone Team A23 Movers Dashboard
# # 2023 ACS PUMS 1-Yr Data
# # Kathleen created 5/30/25
# # Last edited 6/13/25

# # After running the following code in a .py file, you will see some warning/thread messages. You can disregard them. There will also be a message like...
# # "Warning: to view this Streamlit app on a browser, run it with the following command: streamlit run file_location/file_name.py [ARGUMENTS]"
# # Copy and paste that command into your Terminal application to run the dashboard app locally.

# #######################
# # Import libraries
# import streamlit as st # provides the web app framework
# import pandas as pd # for data manipulation
# import plotly.express as px # for choropleth maps

# #######################
# # Page configuration
# st.set_page_config(
#     page_title="DSTP 2025 Team A3 Movers Dashboard",
#     layout="wide",
#     initial_sidebar_state="expanded")

# #######################
# # Load data
# inbound = pd.read_csv('inbound.csv')
# outbound = pd.read_csv('outbound.csv')

# #######################
# # Sidebar
# with st.sidebar:
#     st.title("DSTP '25 Capstone Team A3 Movers Dashboard")
    
#     # Create drop-down menus for filtering data
#     # Choose just inbound or just outbound, if you only want one set of drop-downs to filter both datasets simultaneously
#     sex_list = list(outbound.sex.unique())
#     age_list = list(outbound.age_group.unique())
#     education_list = list(outbound.education.unique())
#     marital_list = list(outbound.marital_status.unique())
    
#     # Establish possible values for each drop-down
#     selected_sex = st.selectbox('Select Sex', options=outbound["sex"].unique())
#     selected_age = st.selectbox('Select Age Group', options=outbound["age_group"].unique())
#     selected_education = st.selectbox('Select Highest Level of Education', options=outbound["education"].unique())
#     selected_marital = st.selectbox('Select Marital Status', options=outbound["marital_status"].unique())

#     # Define how to filter the inbound and outbound datasets based on the drop-down selections
#     df_in_selected = inbound[(inbound["sex"] == selected_sex) & (inbound["age_group"] == selected_age) & (inbound["education"] == selected_education) & (inbound["marital_status"] == selected_marital)]
#     df_in_selected_sorted = df_in_selected.sort_values(by="count", ascending=False)

#     df_out_selected = outbound[(outbound["sex"] == selected_sex) & (outbound["age_group"] == selected_age) & (outbound["education"] == selected_education) & (outbound["marital_status"] == selected_marital)]
#     df_out_selected_sorted = df_out_selected.sort_values(by="count", ascending=False)

# #######################
# # Plots

# # Choropleth map configuration
# def make_choropleth(input_df, input_id, input_column):
#     choropleth = px.choropleth(input_df, locations=input_id, color=input_column, locationmode="USA-states",
#                                color_continuous_scale="reds",
#                                scope="usa",
#                                labels={'count':'Count'}
#                               )
#     return choropleth

# #######################
# # Dashboard Main Panel
# col = st.columns((4, 2.5), gap='medium') # 2 columns with a 4:2.5 ratio

# # First column: choropleth maps showing counts of inbound and outbound movers by state
# with col[0]:
#     st.markdown('#### Count of Inbound Movers by Select Characteristics')
    
#     choropleth = make_choropleth(df_in_selected, 'current_state_code', 'count')
#     st.plotly_chart(choropleth, use_container_width=True)

#     st.markdown('#### Count of Outbound Movers by Select Characteristics')
    
#     choropleth = make_choropleth(df_out_selected, 'previous_state_code', 'count')
#     st.plotly_chart(choropleth, use_container_width=True)

# # Second column: Sorted bar charts showing inbound and outbound mover counts by state
# with col[1]:
#     st.markdown('#### Top Inbound States')

#     st.dataframe(df_in_selected_sorted,
#                  column_order=("current_state", "count"),
#                  hide_index=True,
#                  width=None,
#                  column_config={
#                     "current_state": st.column_config.TextColumn(
#                         "Inbound State",
#                     ),
#                     "count": st.column_config.ProgressColumn(
#                         "Count",
#                         format="compact"
#                      )}
#                  )
    
#     st.markdown('#### Top Outbound States')

#     st.dataframe(df_out_selected_sorted,
#                  column_order=("previous_state", "count"),
#                  hide_index=True,
#                  width=None,
#                  column_config={
#                     "previous_state": st.column_config.TextColumn(
#                         "Outbound State",
#                     ),
#                     "count": st.column_config.ProgressColumn(
#                         "Count",
#                         format="compact"
#                      )}
#                  )
    
#     # Footer with additional information
#     with st.expander('About', expanded=True):
#         st.write('''
#             - :orange[**Data**]: [U.S. Census Bureau. American Community Survey Public Use Microdata Sample (PUMS) 1-Year, 2023. Accessed via the ACS API.](<https://www.census.gov/programs-surveys/acs/microdata/access.html>)
#             - :orange[**Note**]: The inbound and outbound components above are independent of each other. Migration flows are not visualized in this dashboard.
#             - :orange[**Disclaimer**]: The data presented in this dashboard is for educational purposes only and should not be used for decision-making without further analysis.
#             - :orange[**Questions? Suggestions?**] contact DSTP Cohort 6 Capstone Team A3 member, Kathleen Rogers (EWD)
#             ''')

## **End of Streamlit Dashboard Code**